In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from HGQ import shutup
import numpy as np
import keras
import qkeras
from HGQ import to_proxy_model

with shutup:
    from hls4ml.converters import convert_from_keras_model

In [2]:
qkeras_model = keras.models.Sequential([
    qkeras.QActivation('quantized_bits(8, 0, 0, False, alpha=1)'),  # This layer can NOT be removed
    keras.layers.Reshape((28, 28, 1)),
    qkeras.QConv2D(4,
                   (3, 3),
                   kernel_quantizer='quantized_bits(8, 3, alpha=1)',
                   bias_quantizer='quantized_bits(8, 3, alpha=1)',
                   activation='quantized_relu(8, 4)'
                   ),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    qkeras.QConv2D(4,
                   (3, 3),
                   kernel_quantizer='quantized_bits(8, 3, alpha=1)',
                   bias_quantizer='quantized_bits(8, 3, alpha=1)',
                   activation='quantized_relu(8, 4)'
                   ),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    qkeras.QDense(32,
                  kernel_quantizer='quantized_bits(8, 3, alpha=1)',
                  bias_quantizer='quantized_bits(8, 3, alpha=1)',
                  activation='quantized_relu(8, 4)'
                  ),
    qkeras.QDense(10,
                  kernel_quantizer='quantized_bits(8, 3, alpha=1)',
                  bias_quantizer='quantized_bits(8, 3, alpha=1)',
                  activation='quantized_bits(8, 4, alpha=1)'
                  )
])

qkeras_model.build((None, 28, 28, 1))

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train / np.float32(256.0)
x_test = x_test / np.float32(256.0)

In [4]:
opt = keras.optimizers.Adam(0.003)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
qkeras_model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [5]:
qkeras_model.fit(x_train, y_train, epochs=3, batch_size=128, verbose=1, validation_data=(x_test, y_test))

Epoch 1/3
469/469 [==============================] - 8s 13ms/step - loss: 0.4635 - accuracy: 0.8571 - val_loss: 0.1471 - val_accuracy: 0.9543
Epoch 2/3
469/469 [==============================] - 5s 10ms/step - loss: 0.1513 - accuracy: 0.9535 - val_loss: 0.1258 - val_accuracy: 0.9597
Epoch 3/3
469/469 [==============================] - 5s 10ms/step - loss: 0.1190 - accuracy: 0.9641 - val_loss: 0.1071 - val_accuracy: 0.9667


In [6]:
proxy = to_proxy_model(qkeras_model, aggressive=False)

In [7]:
r_qkeras = qkeras_model(x_test).numpy()
r_proxy = proxy(x_test).numpy()

assert np.all(r_qkeras == r_proxy)

In [8]:
hls_conf = {'model': {'Precision': 'fixed<1,0>', 'ReuseFactor': 1}}
hls_model = convert_from_keras_model(proxy, output_dir='/tmp/qkeras_mnist', io_type='io_stream', hls_config=hls_conf)

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: fixed_point_quantizer, layer type: FixedPointQuantizer, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: reshape, layer type: Reshape, input shapes: [[None, 28, 28, 1]], output shape: [None, 28, 28, 1]
Layer name: q_conv2d, layer type: Conv2D, input shapes: [[None, 28, 28, 1]], output shape: [None, 26, 26, 4]
Layer name: fixed_point_quantizer_1, layer type: FixedPointQuantizer, input shapes: [[None, 26, 26, 4]], output shape: [None, 26, 26, 4]
Layer name: activation, layer type: Activation, input shapes: [[None, 26, 26, 4]], output shape: [None, 26, 26, 4]
Layer name: fixed_point_quantizer_2, layer type: FixedPointQuantizer, input shapes: [[None, 26, 26, 4]], output shape: [None, 26, 26, 4]
Layer name: max_pooling2d, layer type: MaxPooling2D, input shapes: [[None, 26, 26, 4]], output shape: [None, 13, 13, 

In [9]:
hls_model.compile()

Writing HLS project


/home/calad/mambaforge/envs/HGQ/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Done


In [ ]:
r_hls = hls_model.predict(x_test)

In [ ]:
assert np.all(r_qkeras == r_hls.reshape(r_qkeras.shape))